In [ ]:
import sys
import os

sys.path.append('./')
sys.path.append('pyoneer/.')

import numpy as np
import pyoneer.model.dirac_stream as mod
from benchmarking.plots.plot_routines import loclocplot
from pyoneer.operators.linear_operator import ToeplitzificationOperator, FRISampling
from scipy.linalg import lstsq
from pyoneer.utils.fri import coeffs_to_matched_diracs
from pyoneer.algorithms.cadzow_denoising import CadzowAlgorithm
import matplotlib.pyplot as plt

# Parameters
K = 8  # int, number of Diracs
beta = 2 # np.array([1, 2, 3, 4, 5])  # np.ndarray, oversampling parameter
M = beta * K  # np.ndarray, bandwidth parameter
P = M  # np.ndarray, parameter P in [1].
N = 2 * M + 1  # np.ndarray, sizes of seeked Fourier series coefficients.
L = 2 * M + 1  # float, number of measurements
period = 1  # float, period of Dirac stream
PSNR = -10  # np.ndarray, peak signal-to-noise ratios.
seed = 4  # int, seed of random number generator for reproducibility.
tol = 1e-9   # float, tolerance for stopping criterion.
eig_tol = 1e-8  # float, tolerance for low-rank approximation if `backend_cadzow` is 'scipy.sparse'.
nb_cadzow_iter = 30  # int, number of iterations in Cadzow denoising (typically smaller than 20).
backend_cadzow = 'scipy'  # str,  backend for low-rank approximation.

# Settings dictionaries used as inputs to algorithms and routines
settings_dirac = {'dirac_count': K, 'sigma_intensity': 0.5, 't_end': (1 - 0.01) * period,
                  't_start': 0.01 * period,
                  'mean_intensity': 0, 'relative_minimal_distance': 0.01,
                  'intensity_distribution': 'lognormal',
                  'seed': seed}
settings_cadzow = {'nb_iter': nb_cadzow_iter, 'rank': K, 'tol': eig_tol, 'backend': backend_cadzow}


# Create Toeplitzification Operator
Tp = ToeplitzificationOperator(P=P, M=M)
settings_cadzow['toeplitz_op'] = Tp



In [ ]:
# Lipschitz constant evaluation 
n_runs = 100
cadzow = CadzowAlgorithm(**settings_cadzow)
lip_const = 0


# Define an hypercube 
radius = 1
center = np.zeros((N, )).astype(np.complex128)

# Sample the hypercube randomly
low = center-radius
diameter = 2 * radius
samples = low + diameter * np.random.sample((n_runs, N))
samples_2 = low + diameter * np.random.sample((n_runs, N))


for n_run in range(n_runs):

    # Generate first set of samples
    fs_hat = cadzow.denoise(samples[n_run])
    fs_hat_2 = cadzow.denoise(samples_2[n_run])
    lip_const+= np.sqrt(np.sum(np.abs(fs_hat_2 - fs_hat)**2)) / np.sqrt(np.sum(np.abs(samples_2[n_run] - samples[n_run])**2))
lip_const /= n_runs
    


In [ ]:
from collections import defaultdict

n_samples = 10000
eig_tol = 1e-8  # float, tolerance for low-rank approximation if `backend_cadzow` is 'scipy.sparse'.
nb_cadzow_iter = 2  # int, number of iterations in Cadzow denoising (typically smaller than 20).
backend_cadzow = 'scipy'  # str,  backend for low-rank approximation.
Ps = np.arange(2, 12, 2)
Ks = np.arange(2, 12)
lip_const_dict = defaultdict()
for P in Ps:
    lip_const_dict[P] = {}
    for K in Ks:
        if K > P:
            continue
        N = 2 * P + 1         
        settings_cadzow = {'nb_iter': nb_cadzow_iter, 'rank': K, 'tol': eig_tol, 'backend': backend_cadzow}

        
        # Create Toeplitzification Operator
        Tp = ToeplitzificationOperator(P=P, M=P)
        settings_cadzow['toeplitz_op'] = Tp

        # Create Cadzow denoiser
        cadzow = CadzowAlgorithm(**settings_cadzow)

        # Sample the hypercube randomly
        radius = 1
        center = np.zeros((N, )).astype(np.complex128)
        low = center-radius
        diameter = 2 * radius

        random_samps = (np.random.sample((n_runs, N)) + 1j * np.random.sample((n_runs, N)))
        random_samps_2 = (np.random.sample((n_runs, N)) + 1j * np.random.sample((n_runs, N)))
        samples = low + diameter * random_samps / np.abs(random_samps)
        samples_2 = low + diameter * random_samps_2 / np.abs(random_samps_2)

        lip_const_vals = []
        for n_run in range(n_runs):
        
            # Generate first set of samples
            fs_hat = cadzow.denoise(samples[n_run])
            fs_hat_2 = cadzow.denoise(samples_2[n_run])
            lip_const_vals.append(np.sqrt(np.sum(np.abs(fs_hat_2 - fs_hat)**2)) / np.sqrt(np.sum(np.abs(samples_2[n_run] - samples[n_run])**2)))
        lip_const_dict[P][K] = lip_const_vals
        print("*************** K = {} - P = {} - Estimated Lip. const {} ********************".format(K, P, np.mean(np.array(lip_const_vals))))

In [ ]:
P = Ps[1]

fig, axs= plt.subplots(1, 4, figsize=(20, 5))

for i, ax in enumerate(axs):
    ax.boxplot([n for n in lip_const_dict[Ps[1+i]].values()], positions=np.arange(2, Ps[1+i] + 1))
    ax.axhline(np.sqrt(Ps[1+i]+1), linestyle='--')

    ax.set_title("$P={}$".format(Ps[1+i]))
    ax.set_xlabel("$K$")
    axs[0].set_ylabel("$H_n$")
    ax.grid(visible=True)
plt.show()

In [ ]:
fig.savefig("lip_const_test.pdf", format="pdf", bbox_inches="tight")
